<a href="https://colab.research.google.com/github/D3Ml4N/Analisis-Exploratorio-de-Datos/blob/main/Automatizacion_taller2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # **TALLER No2:** Clasificación.

## *Carga de datos.*

In [2]:
import sqlite3
import numpy as np
import pandas as pd

conexion = sqlite3.connect('datos_sensores.db')

#Código SQL.
query = """
SELECT tamaño, polaris.sensor_polaris, orion.sensor_orion, antares.sensor_antares, vega.sensor_vega, categoria,
CASE
    WHEN clasificacion.etiqueta = "Positivo" THEN 1
	  WHEN clasificacion.etiqueta = "Negativo" THEN 0
END as Clasificacion_bin
FROM datos_basicos
LEFT JOIN clasificacion on datos_basicos.id = clasificacion.id
LEFT JOIN orion on orion.id = datos_basicos.id
LEFT JOIN vega on datos_basicos.id = vega.id
LEFT JOIN polaris on datos_basicos.id = polaris.id
LEFT JOIN antares on datos_basicos.id = antares.id
"""
dataset = pd.read_sql_query(query,conexion)
dataset.head()

,tamaño,sensor_polaris,sensor_orion,sensor_antares,sensor_vega,categoria,Clasificacion_bin
0,0.02,35.36,19.77,-62.73,42.15,3,1.0
1,0.11,69.78,-23.51,-38.02,-20.64,3,1.0
2,0.85,30.82,86.63,-71.62,75.26,1,0.0
3,0.53,71.92,-44.91,27.17,4.11,1,0.0
4,0.14,-2.91,-66.34,-74.93,-48.04,2,0.0


## *Creación y entrenamiento del modelo*

In [7]:
from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
#from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#import pandas_profiling
from sklearn.metrics import mean_squared_error

X = dataset.drop('Clasificacion_bin', axis=1)
Y = dataset['Clasificacion_bin']
X.index
T = X.query('index<300000')
L = Y.iloc[:300000]
G = X.query('index>=300000')
print(X)

T_train,T_test,L_train,L_test = train_test_split(T, L, test_size=0.1)

#Creación del modelo
modelo = RandomForestClassifier(n_estimators=1500)
modelo.fit(T_train,L_train)

        tamaño  sensor_polaris  sensor_orion  sensor_antares  sensor_vega  \
0         0.02           35.36         19.77          -62.73        42.15   
1         0.11           69.78        -23.51          -38.02       -20.64   
2         0.85           30.82         86.63          -71.62        75.26   
3         0.53           71.92        -44.91           27.17         4.11   
4         0.14           -2.91        -66.34          -74.93       -48.04   
...        ...             ...           ...             ...          ...   
300095    0.61           75.00         75.72          -33.67       -65.18   
300096    0.08          -97.28         -3.33           30.82       -28.75   
300097    0.41           32.14        -48.04          -46.10       -23.84   
300098    0.71           45.71         15.46           64.84       -39.97   
300099    0.40          -65.14         36.78            5.63       -59.10   

        categoria  
0               3  
1               3  
2              

RandomForestClassifier(n_estimators=1500)

In [8]:
T_test

,tamaño,sensor_polaris,sensor_orion,sensor_antares,sensor_vega,categoria
97082,0.96,-0.96,70.57,-44.73,-51.69,3
279199,0.24,7.61,31.50,78.33,10.19,2
97565,0.75,-47.21,10.76,-48.18,55.13,2
247515,0.83,-18.10,42.17,62.20,20.46,1
76981,0.92,81.59,-79.98,-43.17,98.53,2
...,...,...,...,...,...,...
120641,0.76,64.94,-57.44,18.11,-59.18,3
35616,0.86,-86.42,57.14,-32.49,-44.39,1
150240,0.47,-54.98,-92.11,84.94,55.52,3
50389,0.24,-56.61,-54.17,-10.99,-10.53,2


In [9]:
#Predicciones
predicciones = modelo.predict(T_test)
predicciones

array([1., 0., 1., ..., 1., 1., 1.])

In [10]:
accuracy = accuracy_score(L_test,predicciones)
print(f"Presición del modelo: {accuracy:.2f}")

Presición del modelo: 0.91
